In [2]:
# Asked ChatGPT for a baseline and this is it
# The parameters are the below along with 70 landscape parameters with 
    #Index(['DateTime', 'lat', 'lon', 'wdir15', 't15', 'ws15', 'pres15', 'wdir150',
    #       't150', 'ws150', 'pres150', 'wdir250', 't250', 'ws250', 'pres250',
    #       'wdir500', 't500', 'ws500', 'pres500', 'f', 'fg', 'gust_factor'],
    #      dtype='object')

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd, numpy as np, tensorflow as tf

In [4]:
def mean_absolute_percentage_error(y_true, y_pred):
    return tf.reduce_mean(tf.abs((y_true-y_pred) / y_true)) * 100.0

In [98]:
#df = pd.read_feather('E:/Skóli/HÍ/Vélaverkfræði Master HÍ/Lokaverkefni/Data/merged-full-25ms-24hr-28-2-24.feather')
#df = pd.read_feather('E:/Skóli/HÍ/Vélaverkfræði Master HÍ/Lokaverkefni/Data/merged-full-W-Landscape-And-Station-Elevations-25ms-24hr-11-3-24.feather')
df = pd.read_feather('D:\Skóli\lokaverkefni_vel\data\merged-full-W-Landscape-And-Station-Elevations-25ms-24hr-11-3-24.feather')
df = df[df.f < df.fg]
df['gust_factor'] = df.fg / df.f

df_unfolded = df.elevations.apply(pd.Series)

df = pd.concat([df, df_unfolded], axis = 1)

df = df.dropna()
df = df.reset_index(drop = True)

In [99]:
df.iloc[:, -70:] = df.iloc[:, -70:].sub(df.station_elevation, axis = 0)

In [100]:
from sklearn.decomposition import PCA

df_landscape_elevation = df.iloc[:, -70:]

df_landscape_elevation = (df_landscape_elevation - df_landscape_elevation.mean()) / df_landscape_elevation.std()

n_components = 5

pca = PCA(n_components=n_components)
compressed_features = pca.fit_transform(df_landscape_elevation)

compressed_df = pd.DataFrame(data = compressed_features, columns = ['PC' + str(i) for i in range(n_components)])

df  = pd.concat([df, compressed_df], axis = 1)

In [ ]:
def cornerFromCenterLand(row):
    X, Y, d = row.X, row.Y, row.d
    inlandX, inlandY = 520000, 485000

    v = (X - inlandX, )

In [101]:
df.columns

Index([     'X',      'Y',   'time',  'ws_15', 'ws_250', 'ws_500',  'wd_15',
       'wd_250', 'wd_500',   'p_15',
       ...
             65,       66,       67,       68,       69,    'PC0',    'PC1',
          'PC2',    'PC3',    'PC4'],
      dtype='object', length=106)

In [102]:
y = df.gust_factor
X = df[['Ri_01', 'Ri_12', 'X', 'Y', 'station_elevation'] + ['PC' + str(i) for i in range(n_components)]]

# Changing the type of X,y so as to work with Tensorflow
X, y = X.values.astype(np.float32), y.values.astype(np.float32)
scaler = StandardScaler()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
X_train = scaler.fit_transform(X_train)
X_val = scaler.fit_transform(X_val)
X_test = scaler.fit_transform(X_test)

In [103]:
n_units = 128

model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=n_units, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(units=n_units, activation='relu'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(units=n_units, activation='relu'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(units=n_units, activation='relu'),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(units=n_units, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(units=1, activation='linear')
])

In [104]:
model.compile(optimizer='adam', loss=mean_absolute_percentage_error)
# Train the model
model.fit(X_train, y_train, epochs = 100, batch_size = 128, validation_data = (X_val, y_val))

Epoch 1/100
64/64 [==============================] - 5s 12ms/step - loss: 101.5588 - val_loss: 34.8562
Epoch 2/100
64/64 [==============================] - 0s 7ms/step - loss: 61.0498 - val_loss: 15.8429
Epoch 3/100
64/64 [==============================] - 0s 7ms/step - loss: 42.2420 - val_loss: 10.6266
Epoch 4/100
64/64 [==============================] - 0s 8ms/step - loss: 31.0723 - val_loss: 8.6793
Epoch 5/100
64/64 [==============================] - 0s 7ms/step - loss: 24.5305 - val_loss: 8.4546
Epoch 6/100
64/64 [==============================] - 0s 8ms/step - loss: 20.2408 - val_loss: 8.0156
Epoch 7/100
64/64 [==============================] - 0s 6ms/step - loss: 18.1655 - val_loss: 8.4732
Epoch 8/100
64/64 [==============================] - 0s 6ms/step - loss: 16.6498 - val_loss: 8.1666
Epoch 9/100
64/64 [==============================] - 0s 6ms/step - loss: 15.4485 - val_loss: 8.5872
Epoch 10/100
64/64 [==============================] - 0s 7ms/step - loss: 14.5103 - val_loss: 8

In [105]:
# Evaluate the model
mape = model.evaluate(X_test, y_test)
print(f'Model evaluates to: {mape}%')

71/71 [==============================] - 0s 2ms/step - loss: 6.2152
Model evaluates to: 6.2151641845703125%


In [46]:
model.save('./saved_models/nn-12-3-24-14-00.keras')